In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 10.4 MB/s 
     |████████████████████████████████| 1.6 MB 13.9 MB/s 
     |████████████████████████████████| 9.3 MB 23.1 MB/s 
     |████████████████████████████████| 108 kB 70.2 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [12]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [13]:
df.drop(['index'],axis=1,inplace=True)

In [14]:
df_5percent=df[0:3126] 

In [15]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,232,6,1,0,1,...,1,2,4,0,6.319689,0.000000,95232,7.612957,1,1
1,144,3,4,65535,184,184,3,1,1,1,...,1,0,3,0,5.704725,0.000000,225280,5.531738,0,1
2,144,3,4,65535,184,232,4,1,0,1,...,1,1,3,0,6.165941,5.789084,66432,6.002280,1,0
3,144,3,4,65535,184,248,4,1,1,1,...,1,1,3,0,0.000000,0.199479,105472,5.627883,0,1
4,144,3,4,65535,184,264,4,1,0,1,...,1,1,3,0,6.587611,5.338560,983376,6.738243,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3121,144,3,4,65535,184,224,4,1,0,1,...,1,1,3,0,6.281738,6.034156,65024,6.169592,1,0
3122,144,3,4,65535,184,128,5,1,1,1,...,1,0,5,0,6.272931,2.393692,26624,5.830310,0,1
3123,144,3,4,65535,184,248,4,1,1,1,...,1,0,4,0,3.782408,0.000000,125440,5.336778,0,1
3124,144,3,4,65535,184,272,5,1,0,1,...,1,1,4,0,6.216790,4.826679,121624,6.400428,1,0


In [16]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (1502, 69)
class 1: (1624, 69)


In [17]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [18]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     232.0               6.0   
1      144.0   3.0        4.0     65535.0  184.0     184.0               3.0   
2      144.0   3.0        4.0     65535.0  184.0     232.0               4.0   
3      144.0   3.0        4.0     65535.0  184.0     248.0               4.0   
4      144.0   3.0        4.0     65535.0  184.0     264.0               4.0   
...      ...   ...        ...         ...    ...       ...               ...   
3121   144.0   3.0        4.0     65535.0  184.0     224.0               4.0   
3122   144.0   3.0        4.0     65535.0  184.0     128.0               5.0   
3123   144.0   3.0        4.0     65535.0  184.0     248.0               4.0   
3124   144.0   3.0        4.0     65535.0  184.0     272.0               5.0   
3125   144.0   3.0        4.0     65535.0  184.0     248.0               6.0   

      C

In [19]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,3126.000000,3126.000000,3126.000000,3126.000000,3126.000000,3126.000000,3126.000000,3126.000000,3126.000000,3126.0,...,3126.00000,3126.000000,3126.000000,3126.000000,3126.000000,3126.000000,3.126000e+03,3126.000000,3126.000000,3126.000000
mean,162.246961,15.787908,12.193218,65154.847409,194.603967,223.154191,4.653551,0.992003,0.366923,1.0,...,0.99936,1.346129,3.307422,0.158029,4.969281,2.545802,7.450882e+05,6.380208,0.547985,0.519514
std,795.631757,506.998533,324.903086,4891.859501,444.460603,47.775969,1.916480,0.089084,0.482042,0.0,...,0.02529,1.621800,1.135826,0.364827,2.503912,2.690284,4.703812e+06,1.098912,0.497772,0.499699
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.00000,1.000000,3.000000,0.000000,4.349794,0.000000,6.210400e+04,5.704216,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.00000,1.000000,4.000000,0.000000,6.163816,1.628794,1.228800e+05,6.391830,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.00000,2.000000,4.000000,0.000000,6.502834,4.687082,3.112960e+05,7.321731,1.000000,1.000000
max,37008.000000,20050.000000,12851.000000,65535.000000,17744.000000,648.000000,34.000000,1.000000,1.000000,1.0,...,1.00000,31.000000,8.000000,1.000000,7.999859,7.999620,1.223571e+08,7.999983,1.000000,1.000000


In [20]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [21]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [22]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [23]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      58.125
0.0      41.875
dtype: float64

In [24]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      52.5
0.0      47.5
dtype: float64

In [25]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [26]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [27]:
circuit(weights_init, data[0])

tensor(0.10970386, requires_grad=True)

In [28]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [29]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [30]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [31]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [32]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.31998093, requires_grad=False), tensor(-0.24768521, requires_grad=False)], Y = -1
X = [tensor(-0.26607134, requires_grad=False), tensor(-0.16739127, requires_grad=False)], Y = -1
X = [tensor(-0.26353253, requires_grad=False), tensor(-0.21289411, requires_grad=False)], Y =  1
X = [tensor(-0.26276625, requires_grad=False), tensor(-0.22688748, requires_grad=False)], Y =  1
X = [tensor(-0.26110778, requires_grad=False), tensor(-0.25973282, requires_grad=False)], Y = -1


In [33]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [34]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.1077526 | Accuracy: 0.5937500 
Iter:     2 | Cost: 1.0401476 | Accuracy: 0.5750000 
Iter:     3 | Cost: 1.0006386 | Accuracy: 0.5562500 
Iter:     4 | Cost: 1.1275516 | Accuracy: 0.4125000 
Iter:     5 | Cost: 1.4241494 | Accuracy: 0.4125000 
Iter:     6 | Cost: 1.3772955 | Accuracy: 0.4125000 
Iter:     7 | Cost: 1.1675580 | Accuracy: 0.4250000 
Iter:     8 | Cost: 1.0700388 | Accuracy: 0.4500000 
Iter:     9 | Cost: 1.0092854 | Accuracy: 0.4750000 
New best
Iter:    10 | Cost: 0.9803232 | Accuracy: 0.6312500 
Iter:    11 | Cost: 0.9729776 | Accuracy: 0.5875000 
Iter:    12 | Cost: 0.9838768 | Accuracy: 0.5812500 
Iter:    13 | Cost: 0.9789398 | Accuracy: 0.5750000 
Iter:    14 | Cost: 0.9825453 | Accuracy: 0.5750000 
Iter:    15 | Cost: 0.9986954 | Accuracy: 0.5812500 
Iter:    16 | Cost: 1.0465251 | Accuracy: 0.5812500 
Iter:    17 | Cost: 1.1089269 | Accuracy: 0.5812500 
Iter:    18 | Cost: 1.1553738 | Accuracy: 0.5812500 
Iter:    19 | Cost: 1.213161

In [35]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [36]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.150440314004912, Accuracy: 40.0%


In [37]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,-1.0,1.0
1,1.0,1.0
2,-1.0,1.0
3,1.0,-1.0
4,-1.0,1.0
5,1.0,-1.0
6,1.0,1.0
7,1.0,-1.0
8,-1.0,1.0
9,1.0,-1.0


In [38]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         5
         1.0       0.76      0.46      0.57        35

    accuracy                           0.40        40
   macro avg       0.38      0.23      0.29        40
weighted avg       0.67      0.40      0.50        40

0.7619047619047619
0.45714285714285713
0.5714285714285714
0.22857142857142856
